In [92]:
import numpy as np

from skimage import io
import os
import napari

# import dask_image
import dask.array as da
import dask
import zarr

from fractions import Fraction
from skimage.transform import resize

In [ ]:
level = 2

imgs = {
    'R02_ch488':zarr.open('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/R02_R01/R02ch488_to_R01.zarr',mode='r')[level][:],
    'R01_ch561':zarr.open('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/ch561.zarr',mode='r')[level][:],
    'R01_ch640':zarr.open('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/ch640.zarr',mode='r')[level][:],
    'R02_ch640':zarr.open('/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/R02_R01/R02ch640_to_R01.zarr',mode='r')[level][:]
}

In [42]:
voxel_size = (Fraction(3,1),Fraction(65,100),Fraction(65,100))
target_size = (10,10,10)

voxel_size_d = (2**level) * np.asarray(voxel_size)
fractions = voxel_size_d / np.asarray(target_size)

denominators = np.asarray([i.denominator for i in fractions])

In [55]:
# add dark voxel until I can resize the voxels precisely
shape = np.asarray(nuclei.shape)
pad_target = np.ceil(shape / denominators).astype(int) * denominators

In [99]:
outpath = '/mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03'

In [103]:
for key in imgs:
    img = imgs[key]
    pad_img = np.pad(img,tuple((0, i) for i in pad_target - shape), mode='constant', constant_values=0)
    img_adjusted = (resize(pad_img,tuple((pad_img.shape * fractions).astype(int)),preserve_range=True)).astype(np.uint16)
    io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)

/tmp/ipykernel_2602960/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/R01_ch561_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)
/tmp/ipykernel_2602960/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/R01_ch640_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)
/tmp/ipykernel_2602960/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220615_slc17a7_nuc_sma_human_03/R02_ch640_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)


In [ ]:
#